# twa tutorial

In this tutorial, we demonstrate how `twa` can be used for basic applications in chemistry.

We will go through the process of defining an ontology, creating and managing instances of lab objects in Python, and performing SPARQL queries using a triple store. 

We will use Blazegraph as our triple store and demonstrate how to interact with it through a Python-based SPARQL client. 

Let's get started!

## Install necessary packages
Run this cell to install the necessary Python packages if they are not already installed.

In [1]:
!pip install twa docker

## Importing Libraries

Import necessary modules. Note that for a Python script, it is important to include `from __future__ import annotations` at the beginning of the file.


In [2]:
from __future__ import annotations
from twa import BaseOntology, BaseClass, ObjectProperty, DataProperty, TransitiveProperty
from twa import as_range
from twa import PySparqlClient

from typing import Dict
import time

## Step 1: Define the Ontology

### Create a DummyLabOntology class
This class represents the ontology for our lab objects.


In [3]:
class DummyLabOntology(BaseOntology):
    base_url = 'https://dummy.example/kg/'
    namespace = 'dummylab'
    owl_versionInfo = '0.0.1'
    rdfs_comment = 'A dummy ontology for a lab'

### Define Classes and Properties

Define various classes and properties for lab objects, including containers, solutions, and handlers.

In [4]:
class LabObject(BaseClass):
    is_defined_by_ontology = DummyLabOntology
    consistsOf: ConsistsOf

class ChemicalContainer(LabObject):
    is_defined_by_ontology = DummyLabOntology
    locationID: LocationID
    isFilledWith: IsFilledWith

class Vial(ChemicalContainer):
    roundBottom: RoundBottom

class ChemicalSolution(BaseClass):
    is_defined_by_ontology = DummyLabOntology
    preparationTimestamp: PreparationTimestamp
    name: Name

class GlassObject(BaseClass):
    is_defined_by_ontology = DummyLabOntology
    transparency: Transparency

class LiquidHandlerVial(Vial, GlassObject):
    pass

class LiquidHandlerRack(LabObject):
    is_defined_by_ontology = DummyLabOntology
    maximumSlots: MaximumSlots

    @property
    def locate_all_vials(self) -> Dict[str, LiquidHandlerVial]:
        return {next(iter(vial.locationID.range)):vial for vial in self.consistsOf.range if isinstance(vial, LiquidHandlerVial)}

    @property
    def max_slots(self) -> int:
        return list(self.maximumSlots.range)[0]

    @property
    def empty_slots(self) -> list[int]:
        return [i for i in range(self.max_slots) if i not in self.locate_all_vials]

    def init_vial(self, vial: LiquidHandlerVial):
        if bool(vial.locationID.range):
            raise ValueError(f'A locationID {vial.locationID} is already assigned to vial {vial} before initialising in rack {self.instance_iri}')
        vial.locationID = LocationID(range=self.empty_slots[0])
        self.consistsOf.range.add(vial)

class LiquidHandler(LabObject):
    is_defined_by_ontology = DummyLabOntology

    def move_vial_to_rack(self, vial: LiquidHandlerVial, to_rack: LiquidHandlerRack, to_location: int = None):
        if len(to_rack.locate_all_vials) == to_rack.max_slots:
            raise ValueError(f'Rack {to_rack.instance_iri} is already full')
        if bool(vial.locationID.range):
            raise ValueError(f'A locationID {vial.locationID} is already assigned to vial {vial} before adding to rack {to_rack.instance_iri}')
        if to_location is not None:
            if to_location not in range(to_rack.max_slots):
                raise ValueError(f'Location {to_location} is beyond the maximum slots of rack {to_rack.instance_iri}')
            elif to_location not in to_rack.empty_slots:
                raise ValueError(f'Location {to_location} is not empty, already occupied by {to_rack.locate_all_vials.get(to_location)}')
            else:
                vial.locationID = LocationID(range=to_location)
        else:
            vial.locationID = LocationID(range=to_rack.empty_slots[0])
        to_rack.consistsOf.range.add(vial)

    def remove_vial_from_rack(self, vial: LiquidHandlerVial, from_rack: LiquidHandlerRack):
        if vial not in from_rack.locate_all_vials.values():
            raise ValueError(f'Vial {vial} not found at in rack {from_rack.instance_iri}')
        from_rack.consistsOf.range.remove(vial)
        vial.locationID = LocationID()

    def move_vial_within_rack(self, from_location: int, to_location: int, rack: LiquidHandlerRack):
        if from_location not in rack.locate_all_vials:
            raise ValueError(f'No vial found at location {from_location} of rack {rack.instance_iri}')
        vial = rack.locate_all_vials[from_location]
        self.remove_vial_from_rack(vial, rack)
        self.move_vial_to_rack(vial, rack, to_location)

    def move_vial_across_rack(self, vial: LiquidHandlerVial, from_rack: LiquidHandlerRack, to_rack: LiquidHandlerRack):
        self.remove_vial_from_rack(vial, from_rack)
        self.move_vial_to_rack(vial, to_rack)


In [5]:
class ConsistsOf(TransitiveProperty):
    is_defined_by_ontology = DummyLabOntology
    # NOTE LabObject is defined later in the code
    # `from __future__ import annotations` at the beginning of this file allows the forward reference here
    range: as_range(LabObject)

class IsFilledWith(ObjectProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(ChemicalSolution, 0, 1)

In [6]:
class RoundBottom(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(bool, 0, 1)

class LocationID(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(int)

class PreparationTimestamp(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(int, 0, 1)

class Name(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(str, 1, 1)

class Transparency(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(bool, 1, 1)

class MaximumSlots(DataProperty):
    is_defined_by_ontology = DummyLabOntology
    range: as_range(int, 1, 1)

## Step 2: Setup Docker and SPARQL Client

### Start Blazegraph Docker Container
This step spins up a Blazegraph container to serve as our triple store.


In [7]:
import docker
# Connect to Docker using the default socket or the configuration in your environment:
client = docker.from_env()

# Run Blazegraph container
# It returns a Container object that we will need later for stopping it
blazegraph = client.containers.run(
    'ghcr.io/cambridge-cares/blazegraph:1.1.0',
    ports={'8080/tcp': 9999}, # this binds the internal port 8080/tcp to the external port 9998
    detach=True # this runs the container in the background
)

### Define and initialize SPARQL Client

We can also define a custom SPARQL Client to host queries that are commonly used but not covered by OGM.

In [8]:
class LabSparqlClient(PySparqlClient):
    def get_rack_vial_location(self, rack_iris: list):
        st = f"""SELECT ?rack ?vial ?location WHERE {{
                VALUES ?rack {{ <{"> <".join(rack_iris)}> }} .
                ?rack <{ConsistsOf.get_predicate_iri()}> ?vial.
                ?vial <{LocationID.get_predicate_iri()}> ?location.
            }}"""
        print(st)
        return self.perform_query(st)


Connect to the Blazegraph instance using the SPARQL client.

In [9]:
sparql_endpoint = 'http://localhost:9999/blazegraph/namespace/kb/sparql'
sparql_client = LabSparqlClient(sparql_endpoint, sparql_endpoint)

### Export Ontology to Triple Store
Export the defined ontology to the Blazegraph triple store.


In [11]:
DummyLabOntology.export_to_triple_store(sparql_client)

## Step 3: Create and Manage Lab Objects

### Instantiate Lab Objects
Create instances of chemical solutions, vials, and racks.


In [12]:
# Instantiate a chemical solution
chemical_1 = ChemicalSolution(name='water', preparationTimestamp=int(time.time()))

# Instantiate two vials
vial_1 = LiquidHandlerVial(
    rdfs_label='vial_1',
    roundBottom=True,
    transparency=True,
    isFilledWith=chemical_1,
)
vial_2 = LiquidHandlerVial(
    rdfs_label='vial_2',
    roundBottom=False,
    transparency=False,
)

# Instantiate two racks
# each with a different number of slots
# and each containing one of the vials
rack_1 = LiquidHandlerRack(rdfs_label='rack_1', maximumSlots=5)
rack_2 = LiquidHandlerRack(rdfs_label='rack_1', maximumSlots=3)
rack_1.init_vial(vial_1)
rack_2.init_vial(vial_2)

# Instantiate a liquid handler that manages the two racks
liquid_handler = LiquidHandler(consistsOf=[rack_1, rack_2])

### Push Objects to Triple Store
Push the instantiated objects to the triple store. Note the `recursive_depth` is set to -1 to push all triples.


In [13]:
liquid_handler.push_to_kg(
    sparql_client=sparql_client,
    recursive_depth=-1
)

(<Graph identifier=N741e69a974a04e6aa9c7fbf01b2c3593 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N9b8aa2169fc14b5e90b4051cb0f67ac7 (<class 'rdflib.graph.Graph'>)>)

## Step 4: Perform Queries and Operations

### Query Vial Locations
Retrieve the locations of vials in the racks.


In [14]:
sparql_client.get_rack_vial_location([rack_1.instance_iri, rack_2.instance_iri])

SELECT ?rack ?vial ?location WHERE {
                VALUES ?rack { <https://dummy.example/kg/dummylab/LiquidHandlerRack_294d0eb3-86c7-41f8-bc04-f6efaaa19686> <https://dummy.example/kg/dummylab/LiquidHandlerRack_3984a752-6f06-4e9b-ad03-dfb8347cf760> } .
                ?rack <https://dummy.example/kg/dummylab/consistsOf> ?vial.
                ?vial <https://dummy.example/kg/dummylab/locationID> ?location.
            }


[{'rack': 'https://dummy.example/kg/dummylab/LiquidHandlerRack_294d0eb3-86c7-41f8-bc04-f6efaaa19686',
  'vial': 'https://dummy.example/kg/dummylab/LiquidHandlerVial_5158347c-38ed-4b2a-82c9-5fd8051c0be7',
  'location': '0'},
 {'rack': 'https://dummy.example/kg/dummylab/LiquidHandlerRack_3984a752-6f06-4e9b-ad03-dfb8347cf760',
  'vial': 'https://dummy.example/kg/dummylab/LiquidHandlerVial_8689ea09-ee5a-44fa-a19c-6ede2a46c274',
  'location': '0'}]

### Move Vials Between Racks
Perform operations to move vials between racks and update the triple store.


In [15]:
# Move vial_1 from rack_1 to rack_2
liquid_handler.move_vial_across_rack(vial_1, rack_1, rack_2)

# Move vial_2 from rack_2 to rack_1
liquid_handler.move_vial_across_rack(vial_2, rack_2, rack_1)

# Update the triplestore with the new locations of the vials
liquid_handler.push_to_kg(sparql_client=sparql_client, recursive_depth=-1, pull_first=True)

(<Graph identifier=N2f6910c8849a4c149f27ad9e878c2bc2 (<class 'rdflib.graph.Graph'>)>,
 <Graph identifier=N6db2a0e481ae432888cc2d5d18c47469 (<class 'rdflib.graph.Graph'>)>)

### Query Updated Vial Locations
Retrieve the updated locations of vials after the operations.


In [16]:
sparql_client.get_rack_vial_location([rack_1.instance_iri, rack_2.instance_iri])

SELECT ?rack ?vial ?location WHERE {
                VALUES ?rack { <https://dummy.example/kg/dummylab/LiquidHandlerRack_294d0eb3-86c7-41f8-bc04-f6efaaa19686> <https://dummy.example/kg/dummylab/LiquidHandlerRack_3984a752-6f06-4e9b-ad03-dfb8347cf760> } .
                ?rack <https://dummy.example/kg/dummylab/consistsOf> ?vial.
                ?vial <https://dummy.example/kg/dummylab/locationID> ?location.
            }


[{'rack': 'https://dummy.example/kg/dummylab/LiquidHandlerRack_3984a752-6f06-4e9b-ad03-dfb8347cf760',
  'vial': 'https://dummy.example/kg/dummylab/LiquidHandlerVial_5158347c-38ed-4b2a-82c9-5fd8051c0be7',
  'location': '1'},
 {'rack': 'https://dummy.example/kg/dummylab/LiquidHandlerRack_294d0eb3-86c7-41f8-bc04-f6efaaa19686',
  'vial': 'https://dummy.example/kg/dummylab/LiquidHandlerVial_8689ea09-ee5a-44fa-a19c-6ede2a46c274',
  'location': '0'}]

It is clear that the location of vial `https://dummy.example/kg/dummylab/LiquidHandlerVial_5158347c-38ed-4b2a-82c9-5fd8051c0be7` is now changed from one rack to another rack, as well as the internal location in the rack.

## Clean up
### Stop Blazegraph Docker Container

In [17]:
blazegraph.stop()

If one wish to remove the blazegraph container:

In [18]:
blazegraph.remove()